In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
import pickle
from sklearn.model_selection import train_test_split

In [2]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../../data/multiATIS/intent_list.csv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx.items()}

In [3]:
def mapping2idx(df):

    encoded_intent = []
    for intent in list(df.INTENT):
        try:
            encoded_intent.append(intent2idx[intent])
        except:
            print(intent)
            encoded_intent.append(intent2idx['OOI'])
            
        
    df['INTENT_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens)
                encoded_slot += str(slots2idx['O']) + " "
        slots_ID.append(" ".join(encoded_slot.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [4]:
def WWTLE_noise(data,l):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    cnt = 0
    for sample in data.values.tolist():

        CP_length = len(sample[2].split(" "))
        
        if CP_length <= l:
            continue
        else:
            
            limit_index = l

            TEXT = ' '.join(sample[1].split(' ')[:limit_index-1])
            SLOTS = ' '.join(sample[2].split(' ')[:limit_index-1])
         
            augINTENT.append(sample[3])
            augTEXT.append(TEXT)
            augSLOTS.append(SLOTS)
            augID.append(cnt)
            cnt+=1
               
    augPD = pd.DataFrame([augID,augTEXT,augSLOTS,augINTENT],index=['ID','TEXT','SLOTS','INTENT']).T
    
    return augPD

In [8]:
def WWTLE_aug(data_path,out_path,fn,l):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = WWTLE_noise(data,l=l)
    
    aug_data = mapping2idx(aug_data)

    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path + fn,sep='\t',index=False)
    

In [9]:
WWTLE_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/WWTLE_Noise/8word/','test_EN.tsv',l=8)
WWTLE_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/WWTLE_Noise/10word/','test_EN.tsv',l=10)
WWTLE_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/WWTLE_Noise/12word/','test_EN.tsv',l=12)
WWTLE_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/WWTLE_Noise/14word/','test_EN.tsv',l=14)

atis_day_name
atis_day_name
token B-flight
atis_day_name
atis_day_name
token B-flight
atis_day_name
atis_day_name
token B-flight


# WWTLE training data augmentation

In [5]:
def WWTLE_aug(data_path,out_path,fn):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data_8 = WWTLE_noise(data,l=8)
    aug_data_10 = WWTLE_noise(data,l=10)
    aug_data_12 = WWTLE_noise(data,l=12)
    aug_data_14 = WWTLE_noise(data,l=14)
    
    augmented_data = pd.concat([data,aug_data_10,aug_data_12,aug_data_14,aug_data_8])
    
    augmented_data = mapping2idx(augmented_data)
    
    augmented_data = augmented_data.sample(frac=1).reset_index(drop=True)
    augmented_data.to_csv(out_path + fn,sep='\t',index=False)
    

In [6]:
WWTLE_aug('../../data/multiATIS/raw/train_dev_test/train_EN.tsv','../../data/multiATIS/split/train/WWTLE_Augmented/','test_EN.tsv')